In [46]:
from TM1py.Services import TM1Service

#连接Tm1
with TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="") as tm1:
    
    
    # 获取cubes
    cubes=tm1.cubes.get_all()
    print(cubes[0]) 

#打印出cube信息，则连通成功

{"Name": "Sales", "Dimensions@odata.bind": ["Dimensions('Year')", "Dimensions('Month')", "Dimensions('Version')", "Dimensions('Location')", "Dimensions('Business Unit')", "Dimensions('Product')", "Dimensions('Data Source')", "Dimensions('Measure_Sales')"]}


In [47]:
from TM1py.Services import TM1Service

#连接Tm1
with TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") as tm1:
    
    
    # 获取cubes
    cubes=tm1.cubes.get_all()
    print(cubes[0]) 

{"Name": "Exchange Rate", "Dimensions@odata.bind": ["Dimensions('Scenario')", "Dimensions('SA Year')", "Dimensions('SA Month')", "Dimensions('Version')", "Dimensions('Currency From')", "Dimensions('Currency To')", "Dimensions('M Exchange Rate')"], "Rules": "Skipcheck;\nFeedstrings;\n\n['ACT','WIP',{'BOM Rate','EOM Rate','Average Rate'}] = N:if( DimIx('Currency From', !Currency From) = DimIx('Currency To', !Currency To) , 1,Continue);\n\n\n['ACT','WIP',{'BOM Rate','EOM Rate','Average Rate'}] = N:\nDB('Exchange Rate Input', !Scenario, !SA Year, !SA Month, !Version, !Currency From, !M Exchange Rate) \\\nDB('Exchange Rate Input', !Scenario, !SA Year, !SA Month, !Version, !Currency To, !M Exchange Rate);\n\n\n\nFeeders;"}


In [16]:
from TM1py.Services import TM1Service
# connection to TM1 Server
with TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") as tm1:
    # read Process
    p_name='93.Sys Server Cube }tiView.Delete'
    p = tm1.processes.get(p_name)
    

In [18]:
from TM1py.Objects import Process
from TM1py.Services import TM1Service

with TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="") as tm1:
    if tm1.processes.exists(p_name):
        tm1.processes.update(p)
    else:
        tm1.processes.create(p)
            
   

### 迁移TI

In [19]:
from TM1py.Services import TM1Service
from TM1py.Objects import Process
tm1_src=TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") 
tm1_tgr=TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="")
p_name_syn='93.Sys Server Cube }tiView.Delete'
p_syn_src = tm1_src.processes.get(p_name_syn)

if tm1_tgr.processes.exists(p_name_syn):
    tm1_tgr.processes.update(p_syn_src)
else:
    tm1_tgr.processes.create(p_syn_src)

### 迁移Dim

In [25]:
from TM1py.Services import TM1Service
tm1_src = TM1Service(address="localhost", port=6023, ssl=True, user="admin", password="")
tm1_tgr = TM1Service(address="localhost", port=30012, ssl=False, user="admin", password="")
syn_dimension='Year Month'

dimension_src = tm1_src.dimensions.get(dimension_name=
syn_dimension)


#迁移维度
if tm1_tgr.dimensions.exists(syn_dimension):
    tm1_tgr.dimensions.update(dimension_src)
else:    
    tm1_tgr.dimensions.create(dimension_src)
#属性迁移
mdx = "SELECT " \
           "{[}ElementAttributes_"+syn_dimension+"].[}ElementAttributes_"+syn_dimension+"].Members} ON COLUMNS , " \
           "{["+syn_dimension+"].["+syn_dimension+"].Members} ON ROWS " \
           "FROM [}ElementAttributes_"+syn_dimension+"]"
cells = tm1_src.cubes.cells.execute_mdx(mdx, skip_rule_derived_cells=True, skip_zeros=True, element_unique_names=False,
                                        skip_cell_properties=True)

tm1_tgr.cells.write("}ElementAttributes_"+syn_dimension, cells, use_ti=True)

#同步subset暂时不能动态，只能第一次初始化
subsets_names = tm1_src.subsets.get_all_names(
syn_dimension)
for subsets_name in subsets_names:
    subset_src = tm1_src.subsets.get(subset_name=subsets_name, dimension_name=
syn_dimension,private=False)
    #subset = tm1.dimensions.subsets.get(dimension_name='Organisation', subset_name=subset_name, private=True)
    if tm1_tgr.subsets.exists(subsets_name,dimension_name=syn_dimension)==False:
        tm1_tgr.subsets.create(subset=subset_src,private=False)


### 迁移批量Dim (最好多同步几次)

In [105]:
tm1_src = TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123")
tm1_tgr = TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="")
dimension_names = tm1_src.dimensions.get_all_names()


for syn_dimension in dimension_names:
     
    dimension_src = tm1_src.dimensions.get(dimension_name=
    syn_dimension)
    
    try:
        
     #迁移维度
        if tm1_tgr.dimensions.exists(syn_dimension):
            tm1_tgr.dimensions.update(dimension_src)
        else:  
        
            tm1_tgr.dimensions.create(dimension_src)
        #属性迁移
        ###留空
        #同步subset暂时不能动态，只能第一次初始化
        subsets_names = tm1_src.subsets.get_all_names(
        syn_dimension)
        for subsets_name in subsets_names:
            subset_src = tm1_src.subsets.get(subset_name=subsets_name, dimension_name=
        syn_dimension,private=False)
            
            if tm1_tgr.subsets.exists(subsets_name,dimension_name=syn_dimension)==False:
                tm1_tgr.subsets.create(subset=subset_src,private=False)
    except:
        print("本次未迁移维度：",dimension_src.name)
    continue


本次未迁移维度： Account
本次未迁移维度： Location
本次未迁移维度： }Groups


### 迁移Cube

In [104]:
from TM1py.Services import TM1Service
from TM1py.Objects import Process
tm1_src=TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") 
tm1_tgr=TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="")
c_name_syn='Sales'
c_syn_src = tm1_src.cubes.get(c_name_syn)

if tm1_tgr.cubes.exists(c_name_syn):
    tm1_tgr.cubes.delete(c_name_syn)
    tm1_tgr.cubes.create(c_syn_src)
else:
    tm1_tgr.cubes.create(c_syn_src)


### 迁移批量Cube

In [106]:
from TM1py.Services import TM1Service
from TM1py.Objects import Process
tm1_src=TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") 
tm1_tgr=TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="")
c_names_syn=tm1_src.cubes.get_all_names()


for c_name_syn in c_names_syn:
    c_syn_src = tm1_src.cubes.get(c_name_syn)
    try:
        if tm1_tgr.cubes.exists(c_name_syn):
            tm1_tgr.cubes.delete(c_name_syn)
            tm1_tgr.cubes.create(c_syn_src)
        else:
            tm1_tgr.cubes.create(c_syn_src)
    except:
        print("本次未迁移Cube：",c_syn_src.name)
    continue    

本次未迁移Cube： }DimensionProperties


### 迁移视图 (暂时对于11.X到10.2.2版本的视图迁移，只支持公有视图）

In [164]:
from TM1py.Services import TM1Service
from TM1py.Objects import Process
tm1_src=TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") 
tm1_tgr=TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="")
c_name_syn='Sales'

View_private,View_public=tm1_src.cubes.views.get_all(cube_name=c_name_syn)
for view_pub in View_public:
    if tm1_tgr.cubes.views.exists(cube_name=c_name_syn,view_name=view_pub.name)[1]:
        tm1_tgr.cubes.views.delete(cube_name=c_name_syn, view_name=view_pub.name, private=False)
        tm1_tgr.cubes.views.create(view=view_pub, private=False)
    else:   
        tm1_tgr.cubes.views.create(view=view_pub, private=False)
for view_pri in View_private:
    if tm1_tgr.cubes.views.exists(cube_name=c_name_syn,view_name=view_pri.name)[1]:
        tm1_tgr.cubes.views.delete(cube_name=c_name_syn, view_name=view_pri.name, private=True)
        tm1_tgr.cubes.views.create(view=view_pri, private=True)
    else:
        tm1_tgr.cubes.views.create(view=view_pri, private=True)       
        
        

### 迁移批量视图

In [165]:
from TM1py.Services import TM1Service
from TM1py.Objects import Process
tm1_src=TM1Service(address="192.168.1.177", port=30015, ssl=False, user="admin", password="123") 
tm1_tgr=TM1Service(address="192.168.1.155", port=30018, ssl=False, user="admin", password="")
c_names_syn=tm1_src.cubes.get_all_names()

for c_name_syn in c_names_syn:
    View_private,View_public=tm1_src.cubes.views.get_all(cube_name=c_name_syn)
    try:
        for view_pub in View_public:
            if tm1_tgr.cubes.views.exists(cube_name=c_name_syn,view_name=view_pub.name)[1]:
                tm1_tgr.cubes.views.delete(cube_name=c_name_syn, view_name=view_pub.name, private=False)
                tm1_tgr.cubes.views.create(view=view_pub, private=False)
            else:
                tm1_tgr.cubes.views.create(view=view_pub, private=False)

    except:    
        print("本次未迁移公有View：",view_pub.name,"in ",c_name_syn)
    continue     
    try:    
        for view_pri in View_private:
            if tm1_tgr.cubes.views.exists(cube_name=c_name_syn,view_name=view_pri.name)[1]:
                tm1_tgr.cubes.views.delete(cube_name=c_name_syn, view_name=view_pri.name, private=True)
                tm1_tgr.cubes.views.create(view=view_pri, private=True)
            else:
                tm1_tgr.cubes.views.create(view=view_pri, private=True)     
    except:
        print("本次未迁移私有View：",view_pri.name,"in ",c_name_syn)
    continue 

本次未迁移公有View： Default in  Exchange Rate Input
本次未迁移公有View： 检查数据加载 in  General Ledger


In [22]:
cells

{('Year Month Rollup', 'mm/YYYY'): 'Year Month Rollup', ('Year Month Rollup', 'mm/YY'): 'Year Month Rollup', ('Year Month Rollup', 'YYYYMM'): 'Year Month Rollup', ('Year Month Rollup', 'YYYY-MM'): 'Year Month Rollup', ('Year Month Rollup', 'MMM-YY'): 'Year Month Rollup', ('Year Month Rollup', 'MMM YYYY'): 'Year Month Rollup', ('Year Month Rollup', 'MMM YY'): 'Year Month Rollup', ('All Years', 'mm/YYYY'): 'All Years', ('All Years', 'mm/YY'): 'All Years', ('All Years', 'YYYYMM'): 'All Years', ('All Years', 'YYYY-MM'): 'All Years', ('All Years', 'MMM-YY'): 'All Years', ('All Years', 'MMM YYYY'): 'All Years', ('All Years', 'MMM YY'): 'All Years', ('2016', 'mm/YYYY'): '2016', ('2016', 'mm/YY'): '2016', ('2016', 'YYYYMM'): '2016', ('2016', 'YYYY-MM'): '2016', ('2016', 'MMM-YY'): '2016', ('2016', 'MMM YYYY'): '2016', ('2016', 'MMM YY'): '2016', ('2016 M01', 'mm/YYYY'): '01/2016', ('2016 M01', 'mm/YY'): '01/16', ('2016 M01', 'Year'): '2016', ('2016 M01', 'YYYYMM'): '201601', ('2016 M01', 'YYYY

In [26]:
cells

{('Year Month Rollup', 'mm/YYYY'): {'Ordinal': 0, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('Year Month Rollup', 'mm/YY'): {'Ordinal': 1, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('Year Month Rollup', 'YYYYMM'): {'Ordinal': 3, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('Year Month Rollup', 'YYYY-MM'): {'Ordinal': 4, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('Year Month Rollup', 'MMM-YY'): {'Ordinal': 9, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('Year Month Rollup', 'MMM YYYY'): {'Ordinal': 10, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('Year Month Rollup', 'MMM YY'): {'Ordinal': 11, 'Value': 'Year Month Rollup', 'Updateable': 258, 'RuleDerived': False}, ('All Years', 'mm/YYYY'): {'Ordinal': 13, 'Value': 'All Years', 'Updateable': 258, 'RuleDerived': False}, ('All Years', 'mm/YY'): {'Ordinal': 14, 'Value':